# Train dataset preparation

#### dependencies

In [1]:
import os
import sys
import re
import calendar
import pandas as pd
import numpy as np

#### Загружаем данные

In [2]:
path2data = 'c:\\Users\\ds_master\\hackaton'

In [2]:
%%time

if sys.platform=='linux':
    path2data = '/mnt/wind/hac2/'
else:
    path2data = 'c:\\Users\\shvets-py\\Documents\\atms\\final_shvets'

print('importing info')
info = pd.read_csv(os.path.join(path2data, 'ssd_info.csv'))
print('importing state')
state = pd.read_csv(os.path.join(path2data, 'ssd_state.csv'))
print('importing trans (later)')

importing info
importing state
importing trans (later)
Wall time: 479 ms


#### Макропеременные

In [3]:
FORECAST_PERIOD = 'weekofyear2'
FORECAST_AFTER_DATETIME = pd.datetime.strptime('2018-01-12', '%Y-%m-%d')

#### Global functions

In [4]:
def datetime_to_period(dt, period_type):
    if period_type=='month':
        return pd.DatetimeIndex(dt).month
    elif period_type=='weekofyear':
        return pd.DatetimeIndex(dt).weekofyear
    elif period_type=='weekofyear2':
        return (pd.DatetimeIndex(dt).weekofyear + 1) // 2
    else:
        return pd.DatetimeIndex(dt)

In [5]:
def aggdummies(data, dummy_prefix, agg_func):
    return data.filter(like=dummy_prefix).agg(agg_func)
def divide(a, b):
    return np.divide(a, b, out=np.zeros_like(a), where=b!=0)
def days_in_forecast_period(x, period_type):
    if period_type=='month':
        return x.apply(lambda x: calendar.monthrange(x.year, x.month)[1]).mean()
    elif period_type=='weekofyear':
        return 7
    elif period_type=='weekofyear2':
        return 14
    else:
        return 0

In [6]:
with open(os.path.join(path2data, 'ssd_trans.csv')) as f:
    head = f.readline()
atm_files = {}
os.makedirs(os.path.join(path2data, 'by_ids'), exist_ok=True)
for atm_id in range(351):
    atm_files[atm_id] = open(os.path.join(path2data + '/by_ids', 'ssd_trans_' + str(atm_id) + '.csv'), mode='a')
    atm_files[atm_id].write(head)

In [7]:
%%time
num_lines = 0
with open(os.path.join(path2data, 'ssd_trans.csv')) as f:
    print(f.readline())
    
    while True:
        line = f.readline()
        if line == '':
            break
        atm_id = int(line.split(',')[0])
        atm_files[atm_id].write(line)
        num_lines += 1

atm_id,dNDCSessionTime,sCardNumber,nTrRequestCount,nTrReplyCount,nCommandCompletionCount,nBnaAcceptCount,nBnaUserErrorCount,nBnaReturnCount,nBnaVaultCount,nCdmDispenseCount,nCdmDeliveryCount,nCdmSprayDispenseCount,nFrontPrepareCount,nFrontConfirmCount,nReceiptPrintCount,nCardCaptureCount,nSessionErrorCount,dCardRemovalTime,dSesionEndTime,dCdmDeliveryTime,sCdmDeliveryDeliveryStatus,dCdmDispenseTime,sCdmDispenseDispenseStatus,nCdmDispenseDispenseAmount,nCdmDispenseDispenseCurrency,dBnaAcceptTime,sBnaAcceptStatusId,sBnaAcceptStatusText,nBnaAcceptAcceptedAmount,sBnaAcceptAccaptedNotes,sBnaAcceptAcceptedCurrency,nBnaAcceptRefuseCount,nBnaAcceptType,nBnaAcceptComment,nBnaAcceptedNotesCount,sCardRemovalStatus,sTrRequestOpCodes

Wall time: 3min 58s


## Предобработка данных

### trans table

In [8]:
def trans_aggr(x):
    out = {
        'transactions_count': len(x),
        'distinct_days': len(x['dNDCSessionTime'].unique()),
        'days_in_forecast_period': days_in_forecast_period(x['dNDCSessionTime'], FORECAST_PERIOD),
        'avg_card_number': x['sCardNumber'].mean(),
        'avg_request_count': x['nTrRequestCount'].mean(),
        'avg_reply_count': x['nTrReplyCount'].mean(),
        'avg_command_completion_count': x['nCommandCompletionCount'].mean(),
        'avg_bna_accept_count': x['nBnaAcceptCount'].mean(),
        'avg_bna_user_error_count': x['nBnaUserErrorCount'].mean(),
        'avg_bna_return_count': x['nBnaReturnCount'].mean(),
        'avg_bna_vault_count': x['nBnaVaultCount'].mean(),
        'avg_cdm_dispense_count': x['nCdmDispenseCount'].mean(),
        'avg_cdm_delivery_count': x['nCdmDeliveryCount'].mean(),
        'avg_cdm_spray_dispense_count': x['nCdmSprayDispenseCount'].mean(),
        'avg_front_prepare_count': x['nFrontPrepareCount'].mean(),
        'avg_front_confirm_count': x['nFrontConfirmCount'].mean(),
        'avg_receipt_confirm_count': x['nReceiptPrintCount'].mean(),
        'avg_card_capture_count': x['nCardCaptureCount'].mean(),
        'avg_session_error_count': x['nSessionErrorCount'].mean(),
        
        'avg_bna_accept_refuse_count': x['nBnaAcceptRefuseCount'].mean(),
        'avg_bna_accepted_notes_count': x['nBnaAcceptedNotesCount'].mean(),
        'avg_bna_accepted_amount': x['nBnaAcceptAcceptedAmount'].mean(),
        'avg_bna_accepted_currency': x['sBnaAcceptAcceptedCurrency'].mean(),
        'avg_cdm_dispense_amount': x['nCdmDispenseDispenseAmount'].mean(),
        
    }

    out.update(aggdummies(x, 'avg_cdm_delivery_status', 'mean'))
    out.update(aggdummies(x, 'avg_cdm_dispense_status', 'mean'))
    out.update(aggdummies(x, 'avg_dispence_currency', 'mean'))
    out.update(aggdummies(x, 'avg_bna_accept_status', 'mean'))
    
    return pd.Series(out)

In [9]:
def get_trans_features(x):
    x = x.copy()
    x['avg_card_number_distinct_day'] = divide(x['avg_card_number'] * x['transactions_count'], 
                                               x['distinct_days'])
    x['avg_request_count_distinct_day'] = divide(x['avg_request_count'] * x['transactions_count'], 
                                                 x['distinct_days'])
    x['avg_reply_count_distinct_day'] = divide(x['avg_reply_count'] * x['transactions_count'], 
                                               x['distinct_days'])
    x['avg_command_copletion_count_distinct_day'] = divide(x['avg_command_completion_count'] * \
                                                           x['transactions_count'],
                                                           x['distinct_days'])
    x['avg_bna_accept_count_distinct_day'] = divide(x['avg_bna_accept_count'] * x['transactions_count'], 
                                                    x['distinct_days'])
    x['avg_bna_user_error_count_distinct_day'] = divide(x['avg_bna_user_error_count'] * x['transactions_count'], 
                                                        x['distinct_days'])
    x['avg_bna_return_count_distinct_day'] = divide(x['avg_bna_return_count'] * x['transactions_count'], 
                                                    x['distinct_days'])
    x['avg_bna_vault_count_distinct_day'] = divide(x['avg_bna_vault_count'] * x['transactions_count'], 
                                                   x['distinct_days'])
    x['avg_cdm_dispense_count_distinct_day'] = divide(x['avg_cdm_dispense_count'] * x['transactions_count'],
                                                      x['distinct_days'])
    x['avg_cdm_delivery_count_distinct_day'] = divide(x['avg_cdm_delivery_count'] * x['transactions_count'], 
                                                      x['distinct_days'])
    x['avg_cdm_spray_dispense_count_distinct_day'] = divide(x['avg_cdm_spray_dispense_count'] * \
                                                            x['transactions_count'], 
                                                            x['distinct_days'])
    x['avg_front_prepare_count_distinct_day'] = divide(x['avg_front_prepare_count'] * x['transactions_count'], 
                                                       x['distinct_days'])
    x['avg_front_confirm_count_distinct_day'] = divide(x['avg_front_confirm_count'] * x['transactions_count'], 
                                                       x['distinct_days'])
    x['avg_receipt_confirm_count_distinct_day'] = divide(x['avg_receipt_confirm_count'] * x['transactions_count'], 
                                                         x['distinct_days'])
    x['avg_card_capture_count_distinct_day'] = divide(x['avg_card_capture_count'] * x['transactions_count'], 
                                                      x['distinct_days'])
    x['avg_session_error_count_distinct_day'] = divide(x['avg_session_error_count'] * x['transactions_count'], 
                                                       x['distinct_days'])
    x['avg_cdm_delivery_status_unknown_distinct_day'] = divide(x['avg_cdm_delivery_status_unknown'] * \
                                                               x['transactions_count'], 
                                                               x['distinct_days'])
    x['avg_bna_accept_refuse_count_distinct_day'] = divide(x['avg_bna_accept_refuse_count'] * \
                                                           x['transactions_count'], x['distinct_days'])
    x['avg_bna_accepted_notes_count_distinct_day'] = divide(x['avg_bna_accepted_notes_count'] * \
                                                            x['transactions_count'], x['distinct_days'])
    x['avg_bna_accepted_amount_distinct_day'] = divide(x['avg_bna_accepted_amount'] * \
                                                       x['transactions_count'], x['distinct_days'])
    x['avg_bna_accepted_currency_distinct_day'] = divide(x['avg_bna_accepted_currency'] * x['transactions_count'], 
                                                         x['distinct_days'])
    x['avg_cdm_dispense_amount_distinct_day'] = divide(x['avg_cdm_dispense_amount'] * \
                                                       x['transactions_count'], x['distinct_days'])
    x['count_card_number'] = x['avg_card_number'] * x['transactions_count']
    x['count_request_count'] = x['avg_request_count'] * x['transactions_count']
    x['count_reply_count'] = x['avg_reply_count'] * x['transactions_count']
    x['count_command_copletion_count'] = x['avg_command_completion_count'] * x['transactions_count']
    x['count_bna_accept_count'] = x['avg_bna_accept_count'] * x['transactions_count']
    x['count_bna_user_error_count'] = x['avg_bna_user_error_count'] * x['transactions_count']
    x['count_bna_return_count'] = x['avg_bna_return_count'] * x['transactions_count']
    x['count_bna_vault_count'] = x['avg_bna_vault_count'] * x['transactions_count']
    x['count_cdm_dispense_count'] = x['avg_cdm_dispense_count'] * x['transactions_count']
    x['count_cdm_delivery_count'] = x['avg_cdm_delivery_count'] * x['transactions_count']
    x['count_cdm_spray_dispense_count'] = x['avg_cdm_spray_dispense_count'] * x['transactions_count']
    x['count_front_prepare_count'] = x['avg_front_prepare_count'] * x['transactions_count']
    x['count_front_confirm_count'] = x['avg_front_confirm_count'] * x['transactions_count']
    x['count_receipt_confirm_count'] = x['avg_receipt_confirm_count'] * x['transactions_count']
    x['count_card_capture_count'] = x['avg_card_capture_count'] * x['transactions_count']
    x['count_session_error_count'] = x['avg_session_error_count'] * x['transactions_count']
    x['count_cdm_delivery_status_unknown'] = x['avg_cdm_delivery_status_unknown'] * x['transactions_count']
    x['count_bna_accept_refuse_count'] = x['avg_bna_accept_refuse_count'] * x['transactions_count']
    x['count_bna_accepted_notes_count'] = x['avg_bna_accepted_notes_count'] * x['transactions_count']
    x['count_bna_accepted_amount'] = x['avg_bna_accepted_amount'] * x['transactions_count']
    x['count_bna_accepted_currency'] = x['avg_bna_accepted_currency'] * x['transactions_count']
    x['count_cdm_dispense_amount'] = x['avg_cdm_dispense_amount'] * x['transactions_count']
    
    return x

In [10]:
trans_arr = []
import tqdm
for atm_id in tqdm.tqdm_notebook(range(351)):
    trans = pd.read_csv(os.path.join(path2data + '/by_ids', 'ssd_trans_' + str(atm_id) + '.csv'),\
                                                                  dtype={'dBnaAcceptTime': 'object',\
                                                                         'sBnaAcceptStatusId': 'object',\
                                                                         'sBnaAcceptStatusText': 'object',\
                                                                         'sBnaAcceptAccaptedNotes': 'object',\
                                                                         'nBnaAcceptType': 'object',\
                                                                         'nBnaAcceptComment': 'object'})
    trans['dNDCSessionTime'] = pd.to_datetime(trans['dNDCSessionTime'], format="%Y/%m/%d %H:%M:%S")
    trans['year'] = pd.DatetimeIndex(trans['dNDCSessionTime']).year
    trans['month'] = pd.DatetimeIndex(trans['dNDCSessionTime']).month
    trans['weekofyear'] = pd.DatetimeIndex(trans['dNDCSessionTime']).weekofyear
    trans['period'] = datetime_to_period(trans['dNDCSessionTime'], FORECAST_PERIOD)

    trans['sCdmDeliveryDeliveryStatus'] = trans['sCdmDeliveryDeliveryStatus'].fillna('null')
    trans['sCdmDispenseDispenseStatus'] = trans['sCdmDispenseDispenseStatus'].fillna('null')
    trans['nCdmDispenseDispenseCurrency'] = trans['nCdmDispenseDispenseCurrency'].fillna(0).astype(int)
    trans['sBnaAcceptStatusId'].fillna('null', inplace=True)
    trans['sBnaAcceptStatusId'] = trans['sBnaAcceptStatusId'].apply(lambda x: {'null':'null',
                                                                               '0x00500A00':'ok', 
                                                                               '0x00500A01':'init_failed', 
                                                                               '0x00500A02':'start_failed', 
                                                                               '0x00500A03':'error', 
                                                                               '0x00500A04':'unknown', 
                                                                               '0x00500A05':'virtual'}[x])

    trans = pd.get_dummies(trans, columns=['sCdmDeliveryDeliveryStatus'], prefix=['avg_cdm_delivery_status'])
    if 'avg_cdm_delivery_status_unknown' not in trans.columns:
        trans['avg_cdm_delivery_status_unknown'] = 0
    trans = pd.get_dummies(trans, columns=['sCdmDispenseDispenseStatus'], prefix=['avg_cdm_dispense_status'])
    trans = pd.get_dummies(trans, columns=['nCdmDispenseDispenseCurrency'], prefix=['avg_dispence_currency'])
    trans = pd.get_dummies(trans, columns=['sBnaAcceptStatusId'], prefix=['avg_bna_accept_status'])
    trans = trans.groupby(['atm_id', 'year', 'period']).apply(trans_aggr).reset_index()
    trans = get_trans_features(trans)
    trans_arr.append(trans)
    
    
trans = pd.concat(trans_arr)
print(trans.shape)


(8392, 86)


In [11]:
trans = pd.concat(trans_arr)
print(trans.shape)

(8392, 86)


### state table

In [12]:
def get_state_date(x):
    if not pd.isnull(x['dModuleStatusTime']):
        return x['dModuleStatusTime']
    elif not pd.isnull(x['dResetDevicesStart']):
        return x['dResetDevicesStart']
    else:
        return x['dInServiceStart']

In [13]:
state['state_date'] = state.apply(get_state_date, axis=1)
state['state_date'] = pd.to_datetime(state['state_date'], format='%Y/%m/%d %H:%M:%S')
state['year'] = pd.DatetimeIndex(state['state_date']).year
state['month'] = pd.DatetimeIndex(state['state_date']).month
state['weekofyear'] = pd.DatetimeIndex(state['state_date']).weekofyear
state['period'] = datetime_to_period(state['state_date'], FORECAST_PERIOD)

In [14]:
def get_error_date(x, module, suffix=''):
    return x.loc[x[module].isin(['hw error','user error','locked'])]\
            .groupby(['atm_id', 'year', 'period'])\
            .agg({'state_date':np.max})\
            .rename(columns={'state_date' : 'state_date_{suffix}'.format(suffix=suffix)})

In [15]:
cardreader_error_date = get_error_date(state, 'CardReaderStatus', 'cardreader').reset_index()
prr_error_date = get_error_date(state, 'PRRStatus', 'prr').reset_index()
cdm_error_date = get_error_date(state, 'CDMStatus', 'cdm').reset_index()
bna_error_date = get_error_date(state, 'BNAStatus', 'bna').reset_index()

In [16]:
def state_aggr(x):
    out = {
        'status_count': len(x),
        'avg_cdm_error': x['CDMStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_bna_error': x['BNAStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_cardreader_error': x['CardReaderStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_prr_error': x['PRRStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_alm_error': x['ALMStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_ups_error': x['UPSStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_hasp_error': x['HASPStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_connection_error': x['ConnectionStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_mcrw_error': x['MCRWStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_prj_error': x['PRJStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_dep_error': x['DEPStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_key_error': x['KEYStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_monitoring_error': x['MONITORINGStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_license_error': x['LICENSEStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_ndc_error': x['NDCStatus'].isin(['hw error', 'user error', 'locked']).mean(),
        'avg_epp_error': x['EPPStatus'].isin(['hw error', 'user error', 'locked']).mean(),
    }

    return pd.Series(out)

In [17]:
def get_state_features(x):
    x = x.copy()
    x['count_prr_error'] = x['avg_prr_error'] * x['status_count']
    x['count_bna_error'] = x['avg_bna_error'] * x['status_count']
    x['count_cardreader_error'] = x['avg_cardreader_error'] * x['status_count']
    x['count_cdm_error'] = x['avg_cdm_error'] * x['status_count']
    x['count_alm_error'] = x['avg_alm_error'] * x['status_count']
    x['count_ups_error'] = x['avg_ups_error'] * x['status_count']
    x['count_hasp_error'] = x['avg_hasp_error'] * x['status_count']
    x['count_connection_error'] = x['avg_connection_error'] * x['status_count']
    x['count_mcrw_error'] = x['avg_mcrw_error'] * x['status_count']
    x['count_prj_error'] = x['avg_prj_error'] * x['status_count']
    x['count_dep_error'] = x['avg_dep_error'] * x['status_count']
    x['count_key_error'] = x['avg_key_error'] * x['status_count']
    x['count_monitoring_error'] = x['avg_monitoring_error'] * x['status_count']
    x['count_license_error'] = x['avg_license_error'] * x['status_count']
    x['count_ndc_error'] = x['avg_ndc_error'] * x['status_count']
    x['count_epp_error'] = x['avg_epp_error'] * x['status_count']
    x['related_total_errors'] = (x['avg_prr_error'] + x['avg_cardreader_error'] + x['avg_bna_error'] + \
                                 x['avg_cdm_error']) * x['status_count']
    x['non_related_total_errors'] = (x['avg_alm_error'] + x['avg_ups_error'] + x['avg_hasp_error'] + \
                                     x['avg_connection_error'] + x['avg_mcrw_error'] + \
                                     x['avg_prj_error'] + x['avg_dep_error'] + x['avg_key_error'] + \
                                     x['avg_monitoring_error'] + x['avg_license_error'] + x['avg_ndc_error'] + \
                                     x['avg_epp_error']) * x['status_count']
    
    return x

In [18]:
%%time
state = state.groupby(['atm_id', 'year', 'period']).apply(state_aggr).reset_index()
state= get_state_features(state)

Wall time: 25.6 s


### Dataset

In [19]:
dataset = trans.merge(state, on=['atm_id', 'year', 'period'], how='inner')
dataset = dataset.merge(info, on='atm_id', how='inner')
dataset = dataset.merge(bna_error_date, on=['atm_id', 'year', 'period'], how='left')
dataset = dataset.merge(prr_error_date, on=['atm_id', 'year', 'period'], how='left')
dataset = dataset.merge(cdm_error_date, on=['atm_id', 'year', 'period'], how='left')
dataset = dataset.merge(cardreader_error_date, on=['atm_id', 'year', 'period'], how='left')

### Dataset features

In [20]:
def weekofyear_to_date(weekofyear, year):
    fd_dayofweek = pd.datetime(year,1,1).isoweekday()
    if fd_dayofweek>4:
        fd = pd.datetime(year, 1, 1+8-fd_dayofweek)
    else:
        fd = pd.datetime(year, 1, 1-fd_dayofweek+1)
    return fd + pd.to_timedelta((weekofyear-1)*7, 'd')
def floor_date(x, forecast_period):
    if forecast_period=='month':
        return x.apply(lambda x: pd.datetime(year=x.year, month=x.month, day=1)
                       if not pd.isnull(x) else x)
    elif forecast_period=='weekofyear':
        return x.apply(lambda x: (x - pd.to_timedelta(x.dayofweek, 'd')).normalize()
                       if not pd.isnull(x) else x)
    elif forecast_period=='weekofyear2':
        return x.apply(lambda x: weekofyear_to_date(x.weekofyear//2*2+1, x.year)
                    if not pd.isnull(x) else x)
    else:
        return x

def ceil_date(x, forecast_period):
    if forecast_period=='month':
        return floor_date(x.apply(lambda x: (x + pd.to_timedelta(1, 'M')) 
                                  if not pd.isnull(x) else x), forecast_period)
                       #pd.datetime(year=x.year, month=x.month, day=1))
    elif forecast_period=='weekofyear':
        return x.apply(lambda x: (x + pd.to_timedelta(7-x.dayofweek, 'd')).normalize()
                       if not pd.isnull(x) else x)
    elif forecast_period=='weekofyear2':
        return x.apply(lambda x: weekofyear_to_date(x.weekofyear//2*2+3, x.year)
                    if not pd.isnull(x) else x)
    else:
        return x
    
def get_days_since_last_error(x, forecast_period):
    if forecast_period=='month':
        return x.apply(lambda x: (pd.datetime(year=x.year, month=x.month+1, day=1)-x).days
                       if not pd.isnull(x) else x)
    elif forecast_period=='weekofyear':
        return x.apply(lambda x: (pd.datetime(year=x.year, month=x.month, day=x.day+7)-x).days
                       if not pd.isnull(x) else x)
    elif forecast_period=='weekofyear2':
        return 14
    else:
        return 0

In [21]:
def get_dataset_featurs(x):
    x['status_to_transactions'] = x['status_count'] / x['transactions_count']
    x['related_to_transactions'] = x['related_total_errors'] / x['transactions_count']
    x['non_related_to_transactions'] = x['non_related_total_errors'] / x['transactions_count']
    x['related_to_non_related'] = divide(x['related_total_errors'], x['non_related_total_errors'])
    x['total_errors'] = x['related_total_errors'] + x['non_related_total_errors']
    x['related_to_status'] = x['related_total_errors'] / x['status_count']
    x['non_related_to_status'] = x['non_related_total_errors'] / x['status_count']
    x['error_per_day'] = (x['related_total_errors'] + x['non_related_total_errors']) / x['days_in_forecast_period']

    for b in ['cardreader', 'bna', 'cdm', 'prr']:
        x['state_date_{0}'.format(b)] = pd.to_datetime(x['state_date_{0}'.format(b)])
        x['days_since_last_{0}_error'.format(b)] = (ceil_date(x['state_date_{0}'.format(b)], FORECAST_PERIOD) - \
                                                    x['state_date_{0}'.format(b)]).apply(lambda x: x.days)
        x['hours_since_last_{0}_error'.format(b)] = (ceil_date(x['state_date_{0}'.format(b)], FORECAST_PERIOD) - \
                                                    x['state_date_{0}'.format(b)]).apply(lambda x: x / np.timedelta64(1, 'h'))

        x['days_since_last_{0}_error'.format(b)] = x.apply(lambda x: x['days_in_forecast_period'] 
                                                        if pd.isnull(x['days_since_last_{0}_error'.format(b)]) 
                                                        else x['days_since_last_{0}_error'.format(b)], axis=1)
        x['hours_since_last_{0}_error'.format(b)] = x.apply(lambda x: x['days_in_forecast_period']*24
                                                        if pd.isnull(x['hours_since_last_{0}_error'.format(b)]) 
                                                        else x['hours_since_last_{0}_error'.format(b)], axis=1)
        
        x['last_error_{0}_relative'.format(b)] = divide(x['count_{0}_error'.format(b)] * \
                                                        x['days_since_last_{0}_error'.format(b)],
                                                        x['days_in_forecast_period'])
        
        x['{0}_error_by_disticnt_day'.format(b)] = divide(x['count_{0}_error'.format(b)],
                                                          x['distinct_days'])
        
        x['{0}_error_by_day'.format(b)] = divide(x['count_{0}_error'.format(b)], 
                                                  x['days_in_forecast_period'])
    
    return x

In [22]:
dataset = get_dataset_featurs(dataset).sort_values(['atm_id', 'year', 'period'])

#### history

In [23]:
def add_prev(df, periods=3, dropna=False):
    df = df.sort_values(['year','period']).copy()
    columns_to_shift = df.columns.drop(['atm_id', 'year', 'period'])
    
    for i in range(1, periods+1):
        df[[c+'_prev'+str(i) for c in columns_to_shift]] = df[columns_to_shift].shift(i)
        
    if dropna:
        return df.dropna().reset_index(drop=True)
    else:
        return df.reset_index(drop=True)

def get_error_date(x, module, suffix=''):
    return x.loc[x[module].isin(['hw error','user error','locked'])]\
            .groupby(['atm_id', 'year', 'period'])\
            .agg({'state_date':np.max})\
            .rename(columns={'state_date' : 'state_date_{suffix}'.format(suffix=suffix)})
            
def add_gr(df, columns, periods=3):
    df = df.copy()
    for i in range(1, periods+1):
        columns_to_gr = [c+'_gr'+str(i) for c in columns]
        df = df.reindex(columns=df.columns.tolist() + columns_to_gr)
        df[columns_to_gr] = divide(df[columns].values,
                                   df[[c+'_prev'+str(i) for c in columns]].values) - 1
    return df

def get_dataset_with_lag(df, val_features, periods=3):
    prev_features = []
    gr_features = []
    for i in range(1, periods+1):
        prev_features += [x + '_prev' + str(i) for x in val_features]
        gr_features += [x + '_gr' + str(i) for x in val_features]

    new_df = df.groupby(by=['atm_id'], as_index=False).apply(add_prev, periods=periods, dropna=True)    
    new_df = add_gr(new_df, val_features, periods=periods)
    new_df = new_df.replace(to_replace=[np.inf, -np.inf], value=np.nan)
    new_df = new_df.fillna(new_df.mean()).fillna(0)
    return new_df

In [24]:
df = dataset.copy()

In [25]:
cash_in_out_dict = {
    'Cash-Out': 'Cash_Out',
    'Cash-In/Cash-Out': 'Cash_In_Cash_Out'
}
df.cash_in_out = df.cash_in_out.apply(lambda x: cash_in_out_dict[x])
display_type_dict = {
    'Кнопочный': 'Buttons',
    'Сенсорный': 'Sensor'
}
df.display_type = df.display_type.apply(lambda x: display_type_dict[x])
scanner_code_reader_dict = {
    'Нет': 'Nope',
    'Двухмерный': '2D'
}
df.scanner_code_reader = df.scanner_code_reader.apply(lambda x: scanner_code_reader_dict[x])

null_sums = df.isnull().sum()
null_columns = list(null_sums[null_sums != 0].index)
null_columns_float = list(df.dtypes[(df.dtypes.values=='float') & df.dtypes.keys().isin(null_columns)].keys())
null_columns_float

df[null_columns_float] = df.groupby(['model'])[null_columns_float]\
        .transform(lambda x: x.fillna(x.mean()))

In [26]:
df.shape

(8391, 158)

In [27]:
df.age = (pd.to_datetime('2017-01-01', format="%Y/%m/%d") - pd.to_datetime(df.age, format="%Y/%m/%d %H:%M:%S")).apply(lambda x: x.days)

In [28]:
probably_dummies = list(df.dtypes[df.dtypes.values=='object'].keys())
really_dummies = []
oneval_dummies = []
for d in probably_dummies:
    count = len(df[d].unique())
    if count>3:
        print('{0}:'.format(d), df[d].unique()[:4], '... {0} unique values'.format(count))
    else:
        print('{0}:'.format(d), df[d].unique())
    if count>1:
        really_dummies.append(d)
    else:
        oneval_dummies.append(d)
print('-'*50)
print('really_dummies:', really_dummies)
print('-'*50)
print('oneval_dummies:', oneval_dummies)

cash_in_out: ['Cash_Out' 'Cash_In_Cash_Out']
display_type: ['Buttons' 'Sensor']
scanner_code_reader: ['Nope' '2D']
--------------------------------------------------
really_dummies: ['cash_in_out', 'display_type', 'scanner_code_reader']
--------------------------------------------------
oneval_dummies: []


In [29]:
for i in ['state_date_bna', 'state_date_prr', 'state_date_cdm', 'state_date_cardreader', 'cash_in_out', 'display_type', 'scanner_code_reader']:
    if i in really_dummies:
        really_dummies.remove(i)

df.drop(labels=oneval_dummies, inplace=True)

# new lines about continios on val features
oneval_cont = []
for ft in df.columns:
    if df[ft].nunique() < 2 and ft != 'year':
        oneval_cont.append(ft)
df.drop(labels=oneval_cont, inplace=True,axis = 1)
# end new lines about continios on val features


for col in ['state_date_bna', 'state_date_prr', 'state_date_cdm', 'state_date_cardreader']:
    try:
        df.drop(labels=col, inplace=True, axis = 1)
    except:
        pass

In [30]:
bad_cols = list(df.columns[df.isnull().any()].values)
df.drop(bad_cols, axis=1, inplace=True)

In [31]:
df.shape

(8391, 112)

In [32]:
labels = []
targets = ['cdm']
for target in targets:
    label = target + '_label'
    labels.append(label)
    df[label] = (df['avg_' + target + '_error'] > 0).astype(np.int64)

## Предыстория

In [33]:
def add_prev(df, periods=3, dropna=False):
    df = df.sort_values(['year','period']).copy()
    columns_to_shift = df.columns.drop(['atm_id', 'year', 'period'])
    
    for i in range(1, periods+1):
        df[[c+'_prev'+str(i) for c in columns_to_shift]] = df[columns_to_shift].shift(i)
        
    if dropna:
        return df.dropna().reset_index(drop=True)
    else:
        return df.reset_index(drop=True)

In [34]:
def add_gr(df, columns, periods=3):
    df = df.copy()
    for i in range(1, periods+1):
        columns_to_gr = [c+'_gr'+str(i) for c in columns] #df.columns.drop(['year',FORECAST_PERIOD])
        df = df.reindex(columns=df.columns.tolist() + columns_to_gr)
        df[columns_to_gr] = divide(df[columns].values,
                                   df[[c+'_prev'+str(i) for c in columns]].values) - 1
    return df

In [35]:
def get_dataset_with_lag(df, val_features, periods=3):
    prev_features = []
    gr_features = []
    for i in range(1, periods+1):
        prev_features += [x + '_prev' + str(i) for x in val_features]
        gr_features += [x + '_gr' + str(i) for x in val_features]

    new_df = df.groupby(by=['atm_id'], as_index=False).apply(add_prev, periods=periods, dropna=True)    
    new_df = add_gr(new_df, val_features, periods=periods)
    new_df = new_df.replace(to_replace=[np.inf, -np.inf], value=np.nan)
    new_df = new_df.fillna(new_df.mean()).fillna(0)
    return new_df, prev_features, gr_features

In [36]:
def aggdummies(data, dummy_prefix, agg_func):
    return data.filter(like=dummy_prefix).agg(agg_func)
def divide(a, b):
    return np.divide(a, b, out=np.zeros_like(a), where=b!=0)
def days_in_forecast_period(x, period_type):
    if period_type=='month':
        return x.apply(lambda x: calendar.monthrange(x.year, x.month)[1]).mean()
    elif period_type=='weekofyear':
        return 7
    elif period_type=='weekofyear2':
        return 14
    else:
        return 0

In [37]:
really_dummies = ['cash_in_out', 'display_type', 'scanner_code_reader']
dummies_features = ['cash_in_out', 'display_type', 'scanner_code_reader']

In [38]:
%%time
val_features = list(set(df.columns) - set(really_dummies) - set(oneval_dummies) - \
                    set(['atm_id', 'year', 'period']))
new_df, prev_features, gr_features = get_dataset_with_lag(df, val_features, periods=5)
features = val_features + dummies_features + prev_features + gr_features

Wall time: 54.5 s


In [39]:
features = list(pd.Series(features).unique())

#### Берем метки следующего периода как целевые

In [40]:
new_df = new_df.sort_values(['atm_id', 'year', 'period'])
for label in labels:
    new_df[label+'_next'] = new_df.groupby('atm_id')[label].shift(-1)

In [41]:
target_test = []
for at_id in range(351):
    atm_labels = []
    for label in labels:
        atm_labels.insert(len(atm_labels), new_df[new_df['atm_id']==at_id][label])
    target_test.insert(len(target_test), atm_labels)

In [42]:
new_df.shape

(6636, 1199)

#### Приведим фреймы к выложенным на платформе

In [56]:
cols = ['atm_id', 'cardreader_error_by_day', 'hours_since_last_cdm_error', 'avg_cdm_dispense_count_distinct_day', 'status_count', 'avg_cdm_delivery_status_null', 'related_to_status', 'avg_cardreader_error', 'avg_reply_count_distinct_day', 'avg_session_error_count', 'avg_request_count_distinct_day', 'avg_receipt_confirm_count', 'avg_cdm_dispense_amount', 'bna_error_by_disticnt_day', 'non_related_to_status', 'avg_bna_return_count', 'non_related_to_transactions', 'total_errors', 'cdm_label', 'avg_hasp_error', 'avg_card_number_distinct_day', 'avg_dispence_currency_0', 'avg_request_count', 'transactions_count', 'avg_bna_error', 'days_since_last_cdm_error', 'cdm_error_by_disticnt_day', 'last_error_prr_relative', 'avg_bna_return_count_distinct_day', 'cardreader_error_by_disticnt_day', 'avg_cdm_delivery_status_unknown', 'avg_cdm_dispense_status_error', 'avg_cdm_delivery_status_unknown_distinct_day', 'avg_cdm_dispense_status_null', 'avg_front_prepare_count', 'avg_cdm_error', 'prr_error_by_disticnt_day', 'avg_front_prepare_count_distinct_day', 'avg_card_capture_count_distinct_day', 'error_per_day', 'bna_error_by_day', 'related_to_non_related', 'avg_reply_count', 'avg_session_error_count_distinct_day', 'prr_error_by_day', 'last_error_cdm_relative', 'avg_cdm_delivery_status_error', 'hours_since_last_cardreader_error', 'non_related_total_errors', 'avg_cdm_dispense_count', 'avg_cdm_dispense_amount_distinct_day', 'avg_cdm_delivery_status_ok', 'avg_cdm_delivery_count_distinct_day', 'last_error_cardreader_relative', 'related_to_transactions', 'cdm_error_by_day', 'avg_epp_error', 'avg_bna_accept_count_distinct_day', 'avg_cdm_dispense_status_ok', 'avg_card_capture_count', 'avg_card_number', 'avg_cdm_delivery_count', 'avg_command_copletion_count_distinct_day', 'avg_ndc_error', 'days_since_last_prr_error', 'age', 'model', 'avg_dispence_currency_810', 'avg_prr_error', 'avg_bna_accept_status_null', 'hours_since_last_bna_error', 'related_total_errors', 'avg_bna_vault_count_distinct_day', 'avg_bna_vault_count', 'days_since_last_bna_error', 'hours_since_last_prr_error', 'avg_command_completion_count', 'status_to_transactions', 'avg_bna_accept_count', 'avg_receipt_confirm_count_distinct_day', 'days_since_last_cardreader_error', 'avg_cdm_delivery_status_not_taken', 'distinct_days', 'last_error_bna_relative', 'avg_mcrw_error', 'cash_in_out', 'display_type', 'scanner_code_reader', 'cardreader_error_by_day_prev1', 'hours_since_last_cdm_error_prev1', 'avg_cdm_dispense_count_distinct_day_prev1', 'status_count_prev1', 'avg_cdm_delivery_status_null_prev1', 'related_to_status_prev1', 'avg_cardreader_error_prev1', 'avg_reply_count_distinct_day_prev1', 'avg_session_error_count_prev1', 'avg_request_count_distinct_day_prev1', 'avg_receipt_confirm_count_prev1', 'avg_cdm_dispense_amount_prev1', 'bna_error_by_disticnt_day_prev1', 'non_related_to_status_prev1', 'avg_bna_return_count_prev1', 'non_related_to_transactions_prev1', 'total_errors_prev1', 'cdm_label_prev1', 'avg_hasp_error_prev1', 'avg_card_number_distinct_day_prev1', 'avg_dispence_currency_0_prev1', 'avg_request_count_prev1', 'transactions_count_prev1', 'avg_bna_error_prev1', 'days_since_last_cdm_error_prev1', 'cdm_error_by_disticnt_day_prev1', 'last_error_prr_relative_prev1', 'avg_bna_return_count_distinct_day_prev1', 'cardreader_error_by_disticnt_day_prev1', 'avg_cdm_delivery_status_unknown_prev1', 'avg_cdm_dispense_status_error_prev1', 'avg_cdm_delivery_status_unknown_distinct_day_prev1', 'avg_cdm_dispense_status_null_prev1', 'avg_front_prepare_count_prev1', 'avg_cdm_error_prev1', 'prr_error_by_disticnt_day_prev1', 'avg_front_prepare_count_distinct_day_prev1', 'avg_card_capture_count_distinct_day_prev1', 'error_per_day_prev1', 'bna_error_by_day_prev1', 'related_to_non_related_prev1', 'avg_reply_count_prev1', 'avg_session_error_count_distinct_day_prev1', 'prr_error_by_day_prev1', 'last_error_cdm_relative_prev1', 'avg_cdm_delivery_status_error_prev1', 'hours_since_last_cardreader_error_prev1', 'non_related_total_errors_prev1', 'avg_cdm_dispense_count_prev1', 'avg_cdm_dispense_amount_distinct_day_prev1', 'avg_cdm_delivery_status_ok_prev1', 'avg_cdm_delivery_count_distinct_day_prev1', 'last_error_cardreader_relative_prev1', 'related_to_transactions_prev1', 'cdm_error_by_day_prev1', 'avg_epp_error_prev1', 'avg_bna_accept_count_distinct_day_prev1', 'avg_cdm_dispense_status_ok_prev1', 'avg_card_capture_count_prev1', 'avg_card_number_prev1', 'avg_cdm_delivery_count_prev1', 'avg_command_copletion_count_distinct_day_prev1', 'avg_ndc_error_prev1', 'days_since_last_prr_error_prev1', 'age_prev1', 'model_prev1', 'avg_dispence_currency_810_prev1', 'avg_prr_error_prev1', 'avg_bna_accept_status_null_prev1', 'hours_since_last_bna_error_prev1', 'related_total_errors_prev1', 'avg_bna_vault_count_distinct_day_prev1', 'avg_bna_vault_count_prev1', 'days_since_last_bna_error_prev1', 'hours_since_last_prr_error_prev1', 'avg_command_completion_count_prev1', 'status_to_transactions_prev1', 'avg_bna_accept_count_prev1', 'avg_receipt_confirm_count_distinct_day_prev1', 'days_since_last_cardreader_error_prev1', 'avg_cdm_delivery_status_not_taken_prev1', 'distinct_days_prev1', 'last_error_bna_relative_prev1', 'avg_mcrw_error_prev1', 'cardreader_error_by_day_prev2', 'hours_since_last_cdm_error_prev2', 'avg_cdm_dispense_count_distinct_day_prev2', 'status_count_prev2', 'avg_cdm_delivery_status_null_prev2', 'related_to_status_prev2', 'avg_cardreader_error_prev2', 'avg_reply_count_distinct_day_prev2', 'avg_session_error_count_prev2', 'avg_request_count_distinct_day_prev2', 'avg_receipt_confirm_count_prev2', 'avg_cdm_dispense_amount_prev2', 'bna_error_by_disticnt_day_prev2', 'non_related_to_status_prev2', 'avg_bna_return_count_prev2', 'non_related_to_transactions_prev2', 'total_errors_prev2', 'cdm_label_prev2', 'avg_hasp_error_prev2', 'avg_card_number_distinct_day_prev2', 'avg_dispence_currency_0_prev2', 'avg_request_count_prev2', 'transactions_count_prev2', 'avg_bna_error_prev2', 'days_since_last_cdm_error_prev2', 'cdm_error_by_disticnt_day_prev2', 'last_error_prr_relative_prev2', 'avg_bna_return_count_distinct_day_prev2', 'cardreader_error_by_disticnt_day_prev2', 'avg_cdm_delivery_status_unknown_prev2', 'avg_cdm_dispense_status_error_prev2', 'avg_cdm_delivery_status_unknown_distinct_day_prev2', 'avg_cdm_dispense_status_null_prev2', 'avg_front_prepare_count_prev2', 'avg_cdm_error_prev2', 'prr_error_by_disticnt_day_prev2', 'avg_front_prepare_count_distinct_day_prev2', 'avg_card_capture_count_distinct_day_prev2', 'error_per_day_prev2', 'bna_error_by_day_prev2', 'related_to_non_related_prev2', 'avg_reply_count_prev2', 'avg_session_error_count_distinct_day_prev2', 'prr_error_by_day_prev2', 'last_error_cdm_relative_prev2', 'avg_cdm_delivery_status_error_prev2', 'hours_since_last_cardreader_error_prev2', 'non_related_total_errors_prev2', 'avg_cdm_dispense_count_prev2', 'avg_cdm_dispense_amount_distinct_day_prev2', 'avg_cdm_delivery_status_ok_prev2', 'avg_cdm_delivery_count_distinct_day_prev2', 'last_error_cardreader_relative_prev2', 'related_to_transactions_prev2', 'cdm_error_by_day_prev2', 'avg_epp_error_prev2', 'avg_bna_accept_count_distinct_day_prev2', 'avg_cdm_dispense_status_ok_prev2', 'avg_card_capture_count_prev2', 'avg_card_number_prev2', 'avg_cdm_delivery_count_prev2', 'avg_command_copletion_count_distinct_day_prev2', 'avg_ndc_error_prev2', 'days_since_last_prr_error_prev2', 'age_prev2', 'model_prev2', 'avg_dispence_currency_810_prev2', 'avg_prr_error_prev2', 'avg_bna_accept_status_null_prev2', 'hours_since_last_bna_error_prev2', 'related_total_errors_prev2', 'avg_bna_vault_count_distinct_day_prev2', 'avg_bna_vault_count_prev2', 'days_since_last_bna_error_prev2', 'hours_since_last_prr_error_prev2', 'avg_command_completion_count_prev2', 'status_to_transactions_prev2', 'avg_bna_accept_count_prev2', 'avg_receipt_confirm_count_distinct_day_prev2', 'days_since_last_cardreader_error_prev2', 'avg_cdm_delivery_status_not_taken_prev2', 'distinct_days_prev2', 'last_error_bna_relative_prev2', 'avg_mcrw_error_prev2', 'cardreader_error_by_day_prev3', 'hours_since_last_cdm_error_prev3', 'avg_cdm_dispense_count_distinct_day_prev3', 'status_count_prev3', 'avg_cdm_delivery_status_null_prev3', 'related_to_status_prev3', 'avg_cardreader_error_prev3', 'avg_reply_count_distinct_day_prev3', 'avg_session_error_count_prev3', 'avg_request_count_distinct_day_prev3', 'avg_receipt_confirm_count_prev3', 'avg_cdm_dispense_amount_prev3', 'bna_error_by_disticnt_day_prev3', 'non_related_to_status_prev3', 'avg_bna_return_count_prev3', 'non_related_to_transactions_prev3', 'total_errors_prev3', 'cdm_label_prev3', 'avg_hasp_error_prev3', 'avg_card_number_distinct_day_prev3', 'avg_dispence_currency_0_prev3', 'avg_request_count_prev3', 'transactions_count_prev3', 'avg_bna_error_prev3', 'days_since_last_cdm_error_prev3', 'cdm_error_by_disticnt_day_prev3', 'last_error_prr_relative_prev3', 'avg_bna_return_count_distinct_day_prev3', 'cardreader_error_by_disticnt_day_prev3', 'avg_cdm_delivery_status_unknown_prev3', 'avg_cdm_dispense_status_error_prev3', 'avg_cdm_delivery_status_unknown_distinct_day_prev3', 'avg_cdm_dispense_status_null_prev3', 'avg_front_prepare_count_prev3', 'avg_cdm_error_prev3', 'prr_error_by_disticnt_day_prev3', 'avg_front_prepare_count_distinct_day_prev3', 'avg_card_capture_count_distinct_day_prev3', 'error_per_day_prev3', 'bna_error_by_day_prev3', 'related_to_non_related_prev3', 'avg_reply_count_prev3', 'avg_session_error_count_distinct_day_prev3', 'prr_error_by_day_prev3', 'last_error_cdm_relative_prev3', 'avg_cdm_delivery_status_error_prev3', 'hours_since_last_cardreader_error_prev3', 'non_related_total_errors_prev3', 'avg_cdm_dispense_count_prev3', 'avg_cdm_dispense_amount_distinct_day_prev3', 'avg_cdm_delivery_status_ok_prev3', 'avg_cdm_delivery_count_distinct_day_prev3', 'last_error_cardreader_relative_prev3', 'related_to_transactions_prev3', 'cdm_error_by_day_prev3', 'avg_epp_error_prev3', 'avg_bna_accept_count_distinct_day_prev3', 'avg_cdm_dispense_status_ok_prev3', 'avg_card_capture_count_prev3', 'avg_card_number_prev3', 'avg_cdm_delivery_count_prev3', 'avg_command_copletion_count_distinct_day_prev3', 'avg_ndc_error_prev3', 'days_since_last_prr_error_prev3', 'age_prev3', 'model_prev3', 'avg_dispence_currency_810_prev3', 'avg_prr_error_prev3', 'avg_bna_accept_status_null_prev3', 'hours_since_last_bna_error_prev3', 'related_total_errors_prev3', 'avg_bna_vault_count_distinct_day_prev3', 'avg_bna_vault_count_prev3', 'days_since_last_bna_error_prev3', 'hours_since_last_prr_error_prev3', 'avg_command_completion_count_prev3', 'status_to_transactions_prev3', 'avg_bna_accept_count_prev3', 'avg_receipt_confirm_count_distinct_day_prev3', 'days_since_last_cardreader_error_prev3', 'avg_cdm_delivery_status_not_taken_prev3', 'distinct_days_prev3', 'last_error_bna_relative_prev3', 'avg_mcrw_error_prev3', 'cardreader_error_by_day_prev4', 'hours_since_last_cdm_error_prev4', 'avg_cdm_dispense_count_distinct_day_prev4', 'status_count_prev4', 'avg_cdm_delivery_status_null_prev4', 'related_to_status_prev4', 'avg_cardreader_error_prev4', 'avg_reply_count_distinct_day_prev4', 'avg_session_error_count_prev4', 'avg_request_count_distinct_day_prev4', 'avg_receipt_confirm_count_prev4', 'avg_cdm_dispense_amount_prev4', 'bna_error_by_disticnt_day_prev4', 'non_related_to_status_prev4', 'avg_bna_return_count_prev4', 'non_related_to_transactions_prev4', 'total_errors_prev4', 'cdm_label_prev4', 'avg_hasp_error_prev4', 'avg_card_number_distinct_day_prev4', 'avg_dispence_currency_0_prev4', 'avg_request_count_prev4', 'transactions_count_prev4', 'avg_bna_error_prev4', 'days_since_last_cdm_error_prev4', 'cdm_error_by_disticnt_day_prev4', 'last_error_prr_relative_prev4', 'avg_bna_return_count_distinct_day_prev4', 'cardreader_error_by_disticnt_day_prev4', 'avg_cdm_delivery_status_unknown_prev4', 'avg_cdm_dispense_status_error_prev4', 'avg_cdm_delivery_status_unknown_distinct_day_prev4', 'avg_cdm_dispense_status_null_prev4', 'avg_front_prepare_count_prev4', 'avg_cdm_error_prev4', 'prr_error_by_disticnt_day_prev4', 'avg_front_prepare_count_distinct_day_prev4', 'avg_card_capture_count_distinct_day_prev4', 'error_per_day_prev4', 'bna_error_by_day_prev4', 'related_to_non_related_prev4', 'avg_reply_count_prev4', 'avg_session_error_count_distinct_day_prev4', 'prr_error_by_day_prev4', 'last_error_cdm_relative_prev4', 'avg_cdm_delivery_status_error_prev4', 'hours_since_last_cardreader_error_prev4', 'non_related_total_errors_prev4', 'avg_cdm_dispense_count_prev4', 'avg_cdm_dispense_amount_distinct_day_prev4', 'avg_cdm_delivery_status_ok_prev4', 'avg_cdm_delivery_count_distinct_day_prev4', 'last_error_cardreader_relative_prev4', 'related_to_transactions_prev4', 'cdm_error_by_day_prev4', 'avg_epp_error_prev4', 'avg_bna_accept_count_distinct_day_prev4', 'avg_cdm_dispense_status_ok_prev4', 'avg_card_capture_count_prev4', 'avg_card_number_prev4', 'avg_cdm_delivery_count_prev4', 'avg_command_copletion_count_distinct_day_prev4', 'avg_ndc_error_prev4', 'days_since_last_prr_error_prev4', 'age_prev4', 'model_prev4', 'avg_dispence_currency_810_prev4', 'avg_prr_error_prev4', 'avg_bna_accept_status_null_prev4', 'hours_since_last_bna_error_prev4', 'related_total_errors_prev4', 'avg_bna_vault_count_distinct_day_prev4', 'avg_bna_vault_count_prev4', 'days_since_last_bna_error_prev4', 'hours_since_last_prr_error_prev4', 'avg_command_completion_count_prev4', 'status_to_transactions_prev4', 'avg_bna_accept_count_prev4', 'avg_receipt_confirm_count_distinct_day_prev4', 'days_since_last_cardreader_error_prev4', 'avg_cdm_delivery_status_not_taken_prev4', 'distinct_days_prev4', 'last_error_bna_relative_prev4', 'avg_mcrw_error_prev4', 'cardreader_error_by_day_prev5', 'hours_since_last_cdm_error_prev5', 'avg_cdm_dispense_count_distinct_day_prev5', 'status_count_prev5', 'avg_cdm_delivery_status_null_prev5', 'related_to_status_prev5', 'avg_cardreader_error_prev5', 'avg_reply_count_distinct_day_prev5', 'avg_session_error_count_prev5', 'avg_request_count_distinct_day_prev5', 'avg_receipt_confirm_count_prev5', 'avg_cdm_dispense_amount_prev5', 'bna_error_by_disticnt_day_prev5', 'non_related_to_status_prev5', 'avg_bna_return_count_prev5', 'non_related_to_transactions_prev5', 'total_errors_prev5', 'cdm_label_prev5', 'avg_hasp_error_prev5', 'avg_card_number_distinct_day_prev5', 'avg_dispence_currency_0_prev5', 'avg_request_count_prev5', 'transactions_count_prev5', 'avg_bna_error_prev5', 'days_since_last_cdm_error_prev5', 'cdm_error_by_disticnt_day_prev5', 'last_error_prr_relative_prev5', 'avg_bna_return_count_distinct_day_prev5', 'cardreader_error_by_disticnt_day_prev5', 'avg_cdm_delivery_status_unknown_prev5', 'avg_cdm_dispense_status_error_prev5', 'avg_cdm_delivery_status_unknown_distinct_day_prev5', 'avg_cdm_dispense_status_null_prev5', 'avg_front_prepare_count_prev5', 'avg_cdm_error_prev5', 'prr_error_by_disticnt_day_prev5', 'avg_front_prepare_count_distinct_day_prev5', 'avg_card_capture_count_distinct_day_prev5', 'error_per_day_prev5', 'bna_error_by_day_prev5', 'related_to_non_related_prev5', 'avg_reply_count_prev5', 'avg_session_error_count_distinct_day_prev5', 'prr_error_by_day_prev5', 'last_error_cdm_relative_prev5', 'avg_cdm_delivery_status_error_prev5', 'hours_since_last_cardreader_error_prev5', 'non_related_total_errors_prev5', 'avg_cdm_dispense_count_prev5', 'avg_cdm_dispense_amount_distinct_day_prev5', 'avg_cdm_delivery_status_ok_prev5', 'avg_cdm_delivery_count_distinct_day_prev5', 'last_error_cardreader_relative_prev5', 'related_to_transactions_prev5', 'cdm_error_by_day_prev5', 'avg_epp_error_prev5', 'avg_bna_accept_count_distinct_day_prev5', 'avg_cdm_dispense_status_ok_prev5', 'avg_card_capture_count_prev5', 'avg_card_number_prev5', 'avg_cdm_delivery_count_prev5', 'avg_command_copletion_count_distinct_day_prev5', 'avg_ndc_error_prev5', 'days_since_last_prr_error_prev5', 'age_prev5', 'model_prev5', 'avg_dispence_currency_810_prev5', 'avg_prr_error_prev5', 'avg_bna_accept_status_null_prev5', 'hours_since_last_bna_error_prev5', 'related_total_errors_prev5', 'avg_bna_vault_count_distinct_day_prev5', 'avg_bna_vault_count_prev5', 'days_since_last_bna_error_prev5', 'hours_since_last_prr_error_prev5', 'avg_command_completion_count_prev5', 'status_to_transactions_prev5', 'avg_bna_accept_count_prev5', 'avg_receipt_confirm_count_distinct_day_prev5', 'days_since_last_cardreader_error_prev5', 'avg_cdm_delivery_status_not_taken_prev5', 'distinct_days_prev5', 'last_error_bna_relative_prev5', 'avg_mcrw_error_prev5', 'cardreader_error_by_day_gr1', 'hours_since_last_cdm_error_gr1', 'avg_cdm_dispense_count_distinct_day_gr1', 'status_count_gr1', 'avg_cdm_delivery_status_null_gr1', 'related_to_status_gr1', 'avg_cardreader_error_gr1', 'avg_reply_count_distinct_day_gr1', 'avg_session_error_count_gr1', 'avg_request_count_distinct_day_gr1', 'avg_receipt_confirm_count_gr1', 'avg_cdm_dispense_amount_gr1', 'bna_error_by_disticnt_day_gr1', 'non_related_to_status_gr1', 'avg_bna_return_count_gr1', 'non_related_to_transactions_gr1', 'total_errors_gr1', 'cdm_label_gr1', 'avg_card_number_distinct_day_gr1', 'avg_dispence_currency_0_gr1', 'avg_request_count_gr1', 'transactions_count_gr1', 'avg_bna_error_gr1', 'days_since_last_cdm_error_gr1', 'cdm_error_by_disticnt_day_gr1', 'last_error_prr_relative_gr1', 'avg_bna_return_count_distinct_day_gr1', 'cardreader_error_by_disticnt_day_gr1', 'avg_cdm_delivery_status_unknown_gr1', 'avg_cdm_dispense_status_error_gr1', 'avg_cdm_delivery_status_unknown_distinct_day_gr1', 'avg_cdm_dispense_status_null_gr1', 'avg_front_prepare_count_gr1', 'avg_cdm_error_gr1', 'prr_error_by_disticnt_day_gr1', 'avg_front_prepare_count_distinct_day_gr1', 'avg_card_capture_count_distinct_day_gr1', 'error_per_day_gr1', 'bna_error_by_day_gr1', 'related_to_non_related_gr1', 'avg_reply_count_gr1', 'avg_session_error_count_distinct_day_gr1', 'prr_error_by_day_gr1', 'last_error_cdm_relative_gr1', 'avg_cdm_delivery_status_error_gr1', 'hours_since_last_cardreader_error_gr1', 'non_related_total_errors_gr1', 'avg_cdm_dispense_count_gr1', 'avg_cdm_dispense_amount_distinct_day_gr1', 'avg_cdm_delivery_status_ok_gr1', 'avg_cdm_delivery_count_distinct_day_gr1', 'last_error_cardreader_relative_gr1', 'related_to_transactions_gr1', 'cdm_error_by_day_gr1', 'avg_bna_accept_count_distinct_day_gr1', 'avg_cdm_dispense_status_ok_gr1', 'avg_card_capture_count_gr1', 'avg_card_number_gr1', 'avg_cdm_delivery_count_gr1', 'avg_command_copletion_count_distinct_day_gr1', 'avg_ndc_error_gr1', 'days_since_last_prr_error_gr1', 'avg_dispence_currency_810_gr1', 'avg_prr_error_gr1', 'avg_bna_accept_status_null_gr1', 'hours_since_last_bna_error_gr1', 'related_total_errors_gr1', 'avg_bna_vault_count_distinct_day_gr1', 'avg_bna_vault_count_gr1', 'days_since_last_bna_error_gr1', 'hours_since_last_prr_error_gr1', 'avg_command_completion_count_gr1', 'status_to_transactions_gr1', 'avg_bna_accept_count_gr1', 'avg_receipt_confirm_count_distinct_day_gr1', 'days_since_last_cardreader_error_gr1', 'avg_cdm_delivery_status_not_taken_gr1', 'distinct_days_gr1', 'last_error_bna_relative_gr1', 'avg_mcrw_error_gr1', 'cardreader_error_by_day_gr2', 'hours_since_last_cdm_error_gr2', 'avg_cdm_dispense_count_distinct_day_gr2', 'status_count_gr2', 'avg_cdm_delivery_status_null_gr2', 'related_to_status_gr2', 'avg_cardreader_error_gr2', 'avg_reply_count_distinct_day_gr2', 'avg_session_error_count_gr2', 'avg_request_count_distinct_day_gr2', 'avg_receipt_confirm_count_gr2', 'avg_cdm_dispense_amount_gr2', 'bna_error_by_disticnt_day_gr2', 'non_related_to_status_gr2', 'avg_bna_return_count_gr2', 'non_related_to_transactions_gr2', 'total_errors_gr2', 'cdm_label_gr2', 'avg_card_number_distinct_day_gr2', 'avg_dispence_currency_0_gr2', 'avg_request_count_gr2', 'transactions_count_gr2', 'avg_bna_error_gr2', 'days_since_last_cdm_error_gr2', 'cdm_error_by_disticnt_day_gr2', 'last_error_prr_relative_gr2', 'avg_bna_return_count_distinct_day_gr2', 'cardreader_error_by_disticnt_day_gr2', 'avg_cdm_delivery_status_unknown_gr2', 'avg_cdm_dispense_status_error_gr2', 'avg_cdm_delivery_status_unknown_distinct_day_gr2', 'avg_cdm_dispense_status_null_gr2', 'avg_front_prepare_count_gr2', 'avg_cdm_error_gr2', 'prr_error_by_disticnt_day_gr2', 'avg_front_prepare_count_distinct_day_gr2', 'avg_card_capture_count_distinct_day_gr2', 'error_per_day_gr2', 'bna_error_by_day_gr2', 'related_to_non_related_gr2', 'avg_reply_count_gr2', 'avg_session_error_count_distinct_day_gr2', 'prr_error_by_day_gr2', 'last_error_cdm_relative_gr2', 'avg_cdm_delivery_status_error_gr2', 'hours_since_last_cardreader_error_gr2', 'non_related_total_errors_gr2', 'avg_cdm_dispense_count_gr2', 'avg_cdm_dispense_amount_distinct_day_gr2', 'avg_cdm_delivery_status_ok_gr2', 'avg_cdm_delivery_count_distinct_day_gr2', 'last_error_cardreader_relative_gr2', 'related_to_transactions_gr2', 'cdm_error_by_day_gr2', 'avg_bna_accept_count_distinct_day_gr2', 'avg_cdm_dispense_status_ok_gr2', 'avg_card_capture_count_gr2', 'avg_card_number_gr2', 'avg_cdm_delivery_count_gr2', 'avg_command_copletion_count_distinct_day_gr2', 'avg_ndc_error_gr2', 'days_since_last_prr_error_gr2', 'avg_dispence_currency_810_gr2', 'avg_prr_error_gr2', 'avg_bna_accept_status_null_gr2', 'hours_since_last_bna_error_gr2', 'related_total_errors_gr2', 'avg_bna_vault_count_distinct_day_gr2', 'avg_bna_vault_count_gr2', 'days_since_last_bna_error_gr2', 'hours_since_last_prr_error_gr2', 'avg_command_completion_count_gr2', 'status_to_transactions_gr2', 'avg_bna_accept_count_gr2', 'avg_receipt_confirm_count_distinct_day_gr2', 'days_since_last_cardreader_error_gr2', 'avg_cdm_delivery_status_not_taken_gr2', 'distinct_days_gr2', 'last_error_bna_relative_gr2', 'avg_mcrw_error_gr2', 'cardreader_error_by_day_gr3', 'hours_since_last_cdm_error_gr3', 'avg_cdm_dispense_count_distinct_day_gr3', 'status_count_gr3', 'avg_cdm_delivery_status_null_gr3', 'related_to_status_gr3', 'avg_cardreader_error_gr3', 'avg_reply_count_distinct_day_gr3', 'avg_session_error_count_gr3', 'avg_request_count_distinct_day_gr3', 'avg_receipt_confirm_count_gr3', 'avg_cdm_dispense_amount_gr3', 'bna_error_by_disticnt_day_gr3', 'non_related_to_status_gr3', 'avg_bna_return_count_gr3', 'non_related_to_transactions_gr3', 'total_errors_gr3', 'cdm_label_gr3', 'avg_card_number_distinct_day_gr3', 'avg_dispence_currency_0_gr3', 'avg_request_count_gr3', 'transactions_count_gr3', 'avg_bna_error_gr3', 'days_since_last_cdm_error_gr3', 'cdm_error_by_disticnt_day_gr3', 'last_error_prr_relative_gr3', 'avg_bna_return_count_distinct_day_gr3', 'cardreader_error_by_disticnt_day_gr3', 'avg_cdm_delivery_status_unknown_gr3', 'avg_cdm_dispense_status_error_gr3', 'avg_cdm_delivery_status_unknown_distinct_day_gr3', 'avg_cdm_dispense_status_null_gr3', 'avg_front_prepare_count_gr3', 'avg_cdm_error_gr3', 'prr_error_by_disticnt_day_gr3', 'avg_front_prepare_count_distinct_day_gr3', 'avg_card_capture_count_distinct_day_gr3', 'error_per_day_gr3', 'bna_error_by_day_gr3', 'related_to_non_related_gr3', 'avg_reply_count_gr3', 'avg_session_error_count_distinct_day_gr3', 'prr_error_by_day_gr3', 'last_error_cdm_relative_gr3', 'avg_cdm_delivery_status_error_gr3', 'hours_since_last_cardreader_error_gr3', 'non_related_total_errors_gr3', 'avg_cdm_dispense_count_gr3', 'avg_cdm_dispense_amount_distinct_day_gr3', 'avg_cdm_delivery_status_ok_gr3', 'avg_cdm_delivery_count_distinct_day_gr3', 'last_error_cardreader_relative_gr3', 'related_to_transactions_gr3', 'cdm_error_by_day_gr3', 'avg_bna_accept_count_distinct_day_gr3', 'avg_cdm_dispense_status_ok_gr3', 'avg_card_capture_count_gr3', 'avg_card_number_gr3', 'avg_cdm_delivery_count_gr3', 'avg_command_copletion_count_distinct_day_gr3', 'avg_ndc_error_gr3', 'days_since_last_prr_error_gr3', 'avg_dispence_currency_810_gr3', 'avg_prr_error_gr3', 'avg_bna_accept_status_null_gr3', 'hours_since_last_bna_error_gr3', 'related_total_errors_gr3', 'avg_bna_vault_count_distinct_day_gr3', 'avg_bna_vault_count_gr3', 'days_since_last_bna_error_gr3', 'hours_since_last_prr_error_gr3', 'avg_command_completion_count_gr3', 'status_to_transactions_gr3', 'avg_bna_accept_count_gr3', 'avg_receipt_confirm_count_distinct_day_gr3', 'days_since_last_cardreader_error_gr3', 'avg_cdm_delivery_status_not_taken_gr3', 'distinct_days_gr3', 'last_error_bna_relative_gr3', 'avg_mcrw_error_gr3', 'cardreader_error_by_day_gr4', 'hours_since_last_cdm_error_gr4', 'avg_cdm_dispense_count_distinct_day_gr4', 'status_count_gr4', 'avg_cdm_delivery_status_null_gr4', 'related_to_status_gr4', 'avg_cardreader_error_gr4', 'avg_reply_count_distinct_day_gr4', 'avg_session_error_count_gr4', 'avg_request_count_distinct_day_gr4', 'avg_receipt_confirm_count_gr4', 'avg_cdm_dispense_amount_gr4', 'bna_error_by_disticnt_day_gr4', 'non_related_to_status_gr4', 'avg_bna_return_count_gr4', 'non_related_to_transactions_gr4', 'total_errors_gr4', 'cdm_label_gr4', 'avg_card_number_distinct_day_gr4', 'avg_dispence_currency_0_gr4', 'avg_request_count_gr4', 'transactions_count_gr4', 'avg_bna_error_gr4', 'days_since_last_cdm_error_gr4', 'cdm_error_by_disticnt_day_gr4', 'last_error_prr_relative_gr4', 'avg_bna_return_count_distinct_day_gr4', 'cardreader_error_by_disticnt_day_gr4', 'avg_cdm_delivery_status_unknown_gr4', 'avg_cdm_dispense_status_error_gr4', 'avg_cdm_delivery_status_unknown_distinct_day_gr4', 'avg_cdm_dispense_status_null_gr4', 'avg_front_prepare_count_gr4', 'avg_cdm_error_gr4', 'prr_error_by_disticnt_day_gr4', 'avg_front_prepare_count_distinct_day_gr4', 'avg_card_capture_count_distinct_day_gr4', 'error_per_day_gr4', 'bna_error_by_day_gr4', 'related_to_non_related_gr4', 'avg_reply_count_gr4', 'avg_session_error_count_distinct_day_gr4', 'prr_error_by_day_gr4', 'last_error_cdm_relative_gr4', 'avg_cdm_delivery_status_error_gr4', 'hours_since_last_cardreader_error_gr4', 'non_related_total_errors_gr4', 'avg_cdm_dispense_count_gr4', 'avg_cdm_dispense_amount_distinct_day_gr4', 'avg_cdm_delivery_status_ok_gr4', 'avg_cdm_delivery_count_distinct_day_gr4', 'last_error_cardreader_relative_gr4', 'related_to_transactions_gr4', 'cdm_error_by_day_gr4', 'avg_bna_accept_count_distinct_day_gr4', 'avg_cdm_dispense_status_ok_gr4', 'avg_card_capture_count_gr4', 'avg_card_number_gr4', 'avg_cdm_delivery_count_gr4', 'avg_command_copletion_count_distinct_day_gr4', 'avg_ndc_error_gr4', 'days_since_last_prr_error_gr4', 'avg_dispence_currency_810_gr4', 'avg_prr_error_gr4', 'avg_bna_accept_status_null_gr4', 'hours_since_last_bna_error_gr4', 'related_total_errors_gr4', 'avg_bna_vault_count_distinct_day_gr4', 'avg_bna_vault_count_gr4', 'days_since_last_bna_error_gr4', 'hours_since_last_prr_error_gr4', 'avg_command_completion_count_gr4', 'status_to_transactions_gr4', 'avg_bna_accept_count_gr4', 'avg_receipt_confirm_count_distinct_day_gr4', 'days_since_last_cardreader_error_gr4', 'avg_cdm_delivery_status_not_taken_gr4', 'distinct_days_gr4', 'last_error_bna_relative_gr4', 'avg_mcrw_error_gr4', 'cardreader_error_by_day_gr5', 'hours_since_last_cdm_error_gr5', 'avg_cdm_dispense_count_distinct_day_gr5', 'status_count_gr5', 'avg_cdm_delivery_status_null_gr5', 'related_to_status_gr5', 'avg_cardreader_error_gr5', 'avg_reply_count_distinct_day_gr5', 'avg_session_error_count_gr5', 'avg_request_count_distinct_day_gr5', 'avg_receipt_confirm_count_gr5', 'avg_cdm_dispense_amount_gr5', 'bna_error_by_disticnt_day_gr5', 'non_related_to_status_gr5', 'avg_bna_return_count_gr5', 'non_related_to_transactions_gr5', 'total_errors_gr5', 'cdm_label_gr5', 'avg_card_number_distinct_day_gr5', 'avg_dispence_currency_0_gr5', 'avg_request_count_gr5', 'transactions_count_gr5', 'avg_bna_error_gr5', 'days_since_last_cdm_error_gr5', 'cdm_error_by_disticnt_day_gr5', 'last_error_prr_relative_gr5', 'avg_bna_return_count_distinct_day_gr5', 'cardreader_error_by_disticnt_day_gr5', 'avg_cdm_delivery_status_unknown_gr5', 'avg_cdm_dispense_status_error_gr5', 'avg_cdm_delivery_status_unknown_distinct_day_gr5', 'avg_cdm_dispense_status_null_gr5', 'avg_front_prepare_count_gr5', 'avg_cdm_error_gr5', 'prr_error_by_disticnt_day_gr5', 'avg_front_prepare_count_distinct_day_gr5', 'avg_card_capture_count_distinct_day_gr5', 'error_per_day_gr5', 'bna_error_by_day_gr5', 'related_to_non_related_gr5', 'avg_reply_count_gr5', 'avg_session_error_count_distinct_day_gr5', 'prr_error_by_day_gr5', 'last_error_cdm_relative_gr5', 'avg_cdm_delivery_status_error_gr5', 'hours_since_last_cardreader_error_gr5', 'non_related_total_errors_gr5', 'avg_cdm_dispense_count_gr5', 'avg_cdm_dispense_amount_distinct_day_gr5', 'avg_cdm_delivery_status_ok_gr5', 'avg_cdm_delivery_count_distinct_day_gr5', 'last_error_cardreader_relative_gr5', 'related_to_transactions_gr5', 'cdm_error_by_day_gr5', 'avg_bna_accept_count_distinct_day_gr5', 'avg_cdm_dispense_status_ok_gr5', 'avg_card_capture_count_gr5', 'avg_card_number_gr5', 'avg_cdm_delivery_count_gr5', 'avg_command_copletion_count_distinct_day_gr5', 'days_since_last_prr_error_gr5', 'avg_dispence_currency_810_gr5', 'avg_prr_error_gr5', 'avg_bna_accept_status_null_gr5', 'hours_since_last_bna_error_gr5', 'related_total_errors_gr5', 'avg_bna_vault_count_distinct_day_gr5', 'avg_bna_vault_count_gr5', 'days_since_last_bna_error_gr5', 'hours_since_last_prr_error_gr5', 'avg_command_completion_count_gr5', 'status_to_transactions_gr5', 'avg_bna_accept_count_gr5', 'avg_receipt_confirm_count_distinct_day_gr5', 'days_since_last_cardreader_error_gr5', 'avg_cdm_delivery_status_not_taken_gr5', 'distinct_days_gr5', 'last_error_bna_relative_gr5', 'avg_mcrw_error_gr5', 'target', 'period']

In [57]:
trains, tests = new_df[(new_df.year == 2017) & (new_df.period < 24)], new_df[new_df.period == 24]
trains = trains[['atm_id'] + features + ['cdm_label_next', 'period']]
trains.rename(columns={'cdm_label_next': 'target'}, inplace=True)
trains.reset_index(drop=True, inplace=True)
tests = tests[['atm_id'] + features + ['cdm_label_next', 'period']]
tests.rename(columns={'cdm_label_next': 'target'}, inplace=True)
tests.reset_index(drop=True, inplace=True)

trains.target = trains.target.astype(int)
trains = trains.loc[:, cols]

tests = tests.loc[:, cols]
tests = tests.drop('target', axis=1)

In [58]:
trains.shape, tests.shape

((6285, 909), (351, 908))

In [59]:
trains.to_csv(os.path.join(path2data, 'train_data_prepared.csv'), index=False, encoding='utf-8', sep=';')
tests.to_csv(os.path.join(path2data, 'test_data_prepared.csv'), index=False, encoding='utf-8', sep=';')